In [63]:
import pandas as pd
import numpy as np
import re
import os
from glob import glob
PATH = '../Results'
EXT = "*centroids.csv"

In [93]:
nan = np.nan
def using_clump(a):
    return [a[s] for s in np.ma.clump_unmasked(np.ma.masked_where(a.astype(str)==str(np.nan),a))]
def normalize_centroids_in_tracks(tracks_arr):
    for centroids_arr in tracks_arr:
        centroids_arr-=centroids_arr[0]
    return tracks_arr
def str_array_to_float(arr_of_arr_of_str):
    final_mat = []
    for arr_of_arr in arr_of_arr_of_str:
        float_ts = []
        for str in arr_of_arr:
            float_arr = [float(i) for i in re.findall("\d+\.\d+",str)]
            if(len(float_arr)>=2):
                float_ts.append(float_arr)
        if(len(float_ts)>=1):
            final_mat.append(float_ts)
    return np.array(final_mat)
def get_lens(tracks):
    return pd.Series([len(trk) for trk in tracks]).value_counts()
def cut_tracks_and_save(tracks,well_name,ts_len,cut_longer_ts=False,save=True):
    if(cut_longer_ts):
        tracks_final = np.array([trk[:ts_len] for trk in tracks if len(trk)>=ts_len])
    else:
        tracks_final = np.array([trk for trk in tracks if len(trk)==ts_len])
    track_final_norm=normalize_centroids_in_tracks(tracks_final)
    if(save):
        np.save('../npy_files/'+well_name+'.npy',track_final_norm)
    return track_final_norm

In [68]:
def from_csv_path_to_npy(csv_path,well_name,ts_len,cut_longer_ts=False):
    df_str = pd.read_csv(csv_path,index_col=[0])
    splitted = []
    id_well_index = []
    for cell_id, series in df_str.iterrows():
        tracks = np.array(using_clump(np.array(series)))
        for tr in tracks:
            splitted.append(tr)
    tracks_str = np.array(splitted) 
    print("tracks_str shape: ",tracks_str.shape)
    tracks = str_array_to_float(tracks_str)
    cut_tracks_and_save(tracks,well_name,10,False)

In [80]:
def from_results_folder_PATH_to_arrays():
    all_tracks = []
    wells = []
    all_csv_files = [file
                 for path, subdir, files in os.walk(PATH)
                 for file in glob(os.path.join(path, EXT))]
    for file in all_csv_files:
        file_name = file.split('_')
        well_name = file_name[1]
        df_str = pd.read_csv(file,index_col=[0])
        splitted = []
        id_well_index = []
        for cell_id, series in df_str.iterrows():
            tracks = np.array(using_clump(np.array(series)))
            for tr in tracks:
                splitted.append(tr)
        tracks_str = np.array(splitted) 
        print("tracks_str shape: ",tracks_str.shape)
        tracks = str_array_to_float(tracks_str)
        all_tracks.append(tracks)
        wells.append(well_name)
    return all_tracks,wells

In [ ]:
all_tracks,wells = from_results_folder_PATH_to_arrays()

In [107]:
all = []
labels = []
for well_name,tracks in zip(wells,all_tracks):
    tracks_norm_cut = cut_tracks_and_save(tracks,well_name,12,True)
    all.append(tracks_norm_cut)
    labels.append(np.repeat(well_name,len(tracks_norm_cut)))

results_tracks = np.vstack(all)
results_labels = np.concatenate(labels)


In [108]:
len(results_tracks)

3705

In [109]:
np.save('../npy_files/tracks.npy',results_tracks)
np.save('../npy_files/labels.npy',results_labels)

In [ ]:
all_csv_files = [file
                 for path, subdir, files in os.walk(PATH)
                 for file in glob(os.path.join(path, EXT))]
print(all_csv_files)

In [ ]:
for file in all_csv_files:
    file_name = file.split('_')
    well_name = file_name[1]
    view_name = file_name[2]
    crop_name = file_name[4]
    print(f'well:{well_name} , view:{view_name}, crop:{crop_name}')

In [69]:
path = '/Users/amosavni/university/DeepLearningWorkshop/deepcell_mod/DL-WORKSHOP/results/Results_D2_4_crop_0_start_2_2022-08-25_11-10-58/centroids.csv'
from_csv_path_to_npy(path,'D2_4_crop_0',10,True)

tracks_str shape:  (1502,)


/var/folders/s6/khx_n6gj1fgfgkzsbv62f8sw0000gn/T/ipykernel_4293/1242426391.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tracks = np.array(using_clump(np.array(series)))
/var/folders/s6/khx_n6gj1fgfgkzsbv62f8sw0000gn/T/ipykernel_4293/1242426391.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tracks_str = np.array(splitted)
/var/folders/s6/khx_n6gj1fgfgkzsbv62f8sw0000gn/T/ipykernel_4293/1443410114.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with d

### path to a centroids.csv
comment out mac/windows file depending on os you are using

In [10]:
df_t = pd.read_csv('/Users/amosavni/university/DeepLearningWorkshop/deepcell_mod/DL-WORKSHOP/results/Results_D2_4_crop_0_start_2_2022-08-25_11-10-58/centroids.csv',index_col=[0])

# strings df to array of array of strings

In [12]:
splitted = []
id_well_index = []
for cell_id, series in df_t.iterrows():
    tracks = np.array(using_clump(np.array(series)))
    for tr in tracks:
        splitted.append(tr)

/var/folders/s6/khx_n6gj1fgfgkzsbv62f8sw0000gn/T/ipykernel_4293/2767955733.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tracks = np.array(using_clump(np.array(series)))


In [13]:
tracks_str = np.array(splitted) 
print(tracks_str.shape)

(1502,)


/var/folders/s6/khx_n6gj1fgfgkzsbv62f8sw0000gn/T/ipykernel_4293/339763255.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tracks_str = np.array(splitted)


In [15]:
tracks = str_array_to_float(tracks_str)

/var/folders/s6/khx_n6gj1fgfgkzsbv62f8sw0000gn/T/ipykernel_4293/1399472887.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(final_mat)


In [37]:
print(get_lens(tracks))

1     429
2     215
4     137
3     134
5      95
11     64
6      56
7      55
10     51
8      42
9      42
12     37
15     18
13     18
14     18
16     14
17      9
20      9
24      7
21      7
19      6
26      5
29      4
23      3
28      3
22      3
27      3
18      2
30      1
25      1
37      1
33      1
40      1
36      1
dtype: int64


In [16]:
tracks_final = np.array([trk for trk in tracks if len(trk)==10])

In [18]:
track_final_norm=normalize_centroids_in_tracks(tracks_final)

In [21]:
np.save('track_final_norm.npy',track_final_norm)